In [ ]:
%pip install sqlglot

In [ ]:
import sqlglot
from sqlglot import parse_one, exp
from sqlglot.lineage import lineage
from sqlglot.schema import MappingSchema
from sqlglot.dialects import Snowflake
from sqlglot.optimizer import optimize

In [ ]:
optimized = optimize(parse_one(sql, dialect=Snowflake), dialect=Snowflake)

for predicate in optimized.find_all(exp.Where):
    print(predicate)


In [ ]:
for col in predicate.find_all(exp.Condition):
    print(col)

In [ ]:
col

In [ ]:
node = lineage(
    "a",
    "SELECT a FROM z",
)

In [ ]:
sql = """
with a as (
    select 
       id,
       original as col1, 
       col2
    from 
       t1
)
select
    col2 as col2_renamed
from
  a
where a.col1 = 3902
and id = 123
"""
node = lineage(
    "ID",
    sql,
    dialect=Snowflake
)

# source_column_lineage = []
# start = node.name

# source_column_lineage.append(start)

# next_downstream = node.downstream[0]
# while True:
#     source_column_lineage.append(next_downstream.name)
#     if len(next_downstream.downstream) == 0:
#         break
#     next_downstream = next_downstream.downstream[0]

# source_column_lineage

In [ ]:
sql = """
SELECT 
  LOWER(sc.STOREFRONT_ID) AS storefront_id, 
  c.concept AS pricing 
FROM 
  TKWW_ODS.TK_STOREFRONT.STOREFRONT_CONCEPT sc 
  JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT c ON sc.CONCEPT_ID = c.CONCEPT_id 
  JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT_HIERARCHY ch ON sc.CONCEPT_ID = ch.CONCEPT_id 
  JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT_SCHEME_MAPPING csm ON sc.CONCEPT_ID = csm.CONCEPT_id 
  JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT_SCHEME cs ON csm.CONCEPT_SCHEME_ID = cs.CONCEPT_SCHEME_ID 
  AND cs.class IS NOT NULL 
  JOIN (
    SELECT 
      DISTINCT LOWER(STOREFRONT_ID) AS storefront_id 
    FROM 
      tkww_dw.tk_dim.dim_storefront_listing s 
    WHERE 
      sales_profile_status_code = 'LIVE' -- AND CATEGORY_CODE = 'REC'
      ) s ON sc.STOREFRONT_ID = s.STOREFRONT_ID 
WHERE 
  ch.parent_concept = 'Price Range'

"""

In [ ]:
sql = """
WITH tk_leads as (
  SELECT 
    distinct 'TK' AS country_code, 
    -- m.member_id AS member_id,
    lower(dl.anonymous_id) as anonymous_id, 
    mp.vendor_id, 
    CASE WHEN dl.is_direct = 'true' THEN 1 ELSE 0 END AS direct_flag, 
    (1 - direct_flag) AS indirect_flag, 
    l.lead_create_date_key AS lead_day_id, 
    CASE WHEN l.INITIAL_VENDOR_RESPONSE_TIME_MINS <= 2880 THEN 1 ELSE 0 end AS vendor_response_48_flag, 
    CASE WHEN l.INITIAL_VENDOR_RESPONSE_TIME_MINS <= 2880 
    AND l.INITIAL_BRIDE_RESPONSE_TIME_MINS <= 2880 THEN 1 ELSE 0 end AS rtl_flag 
  FROM 
    tkww_dw.tk_fact.f_digital_lead l 
    JOIN TKWW_DW.TK_DIM.DIM_LEAD dl ON l.lead_key = dl.lead_key 
    JOIN TKWW_DW.TK_DIM.DIM_CATEGORY c ON l.category_key = c.category_key 
    JOIN tkww_dw.tk_dim.dim_member m ON l.member_key = m.member_key 
    JOIN TKWW_DW.TK_DIM.DIM_STOREFRONT s ON s.storefront_key = l.storefront_key 
    join GPDW.BODAS_DW.MAP_TK_LOCATION_VENDOR mp ON s.location_id = mp.tk_location_id --AND mp.TK_COUNTRY_CODE = 'US' 
  where 
    l.lead_create_date_key between 20221128 
    and 20221211 
    and dl.anonymous_id is not null
) 
select 
  country_code, 
  anonymous_id, 
  vendor_id, 
  1 as bhva_key, 
  lead_day_id as bhva_date, 
  0.07 as bhva_score 
from 
  tk_leads 
where 
  direct_flag = 1 
union 
select 
  country_code, 
  anonymous_id, 
  vendor_id, 
  2 as bhva_key, 
  lead_day_id as bhva_date, 
  0 as bhva_score 
from 
  tk_leads 
where 
  indirect_flag = 1 
union 
select 
  country_code, 
  anonymous_id, 
  vendor_id, 
  3 as bhva_key, 
  lead_day_id as bhva_date, 
  0.09 as bhva_score 
from 
  tk_leads 
where 
  vendor_response_48_flag = 1 
union 
select 
  country_code, 
  anonymous_id, 
  vendor_id, 
  4 as bhva_key, 
  lead_day_id as bhva_date, 
  0.16 as bhva_score 
from 
  tk_leads 
where 
  rtl_flag = 1

"""

In [ ]:
sql = """
with qhe as (
    select 
        * 
    from 
        query_history_narrowed q
    where start_time between $query_start_time_min and $query_start_time_max
),
query_table_modified as (
    select 
        *
    from object_modified_by_query o
    where query_start_time between $query_start_time_min and $query_start_time_max
)
-- enriched as (
    select
    qhe.col1,
    qom.*

    from qhe
  left join
        query_table_modified qom
        on qhe.query_id = qom.query_id
"""

In [ ]:
sql = """

        SELECT 
          LOWER(sc.STOREFRONT_ID) AS storefront_id,
          c.concept AS pricing
        FROM TKWW_ODS.TK_STOREFRONT.STOREFRONT_CONCEPT sc 
        JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT c ON sc.CONCEPT_ID = c.CONCEPT_id
        JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT_HIERARCHY ch ON sc.CONCEPT_ID = ch.CONCEPT_id
        JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT_SCHEME_MAPPING csm ON sc.CONCEPT_ID = csm.CONCEPT_id
        JOIN TKWW_ODS.TK_TAXONOMY.CONCEPT_SCHEME cs ON csm.CONCEPT_SCHEME_ID = cs.CONCEPT_SCHEME_ID AND cs.class IS NOT NULL
        JOIN (        		
        	SELECT 
        		DISTINCT LOWER(STOREFRONT_ID) AS storefront_id 
        	FROM tkww_dw.tk_dim.dim_storefront_listing s         		
        	WHERE sales_profile_status_code = 'LIVE' 
        	  -- AND CATEGORY_CODE = 'REC'
        	  ) s ON sc.STOREFRONT_ID = s.STOREFRONT_ID 
        WHERE ch.parent_concept = 'Price Range'
"""



In [ ]:
{
   "filter_predicates":[
      {
         "where_clause":"WHERE \"s\".\"sales_profile_status_code\" = \\'LIVE\\'",
         "filter_columns":[
            {
               "table":"s",
               "column":"sales_profile_status_code"
            }
         ]
      }
   ],
   "aliases":{
      "sc":"storefront_concept",
      "c":"concept",
      "ch":"concept_hierarchy",
      "csm":"concept_scheme_mapping",
      "cs":"concept_scheme",
      "s":"dim_storefront_listing"
   },
   "num_optimized_scopes":2
}

In [ ]:
alias.parent.name

In [ ]:
optimized = optimize(parse_one(sql, dialect=Snowflake), dialect=Snowflake)
scopes = traverse_scope(optimized)
print(f"number of scopes: {len(scopes)}")
preds = []
for scope in traverse_scope(optimized):
    # print(f"scope: {scope}")
    aliases = {}
    for alias in scope.find_all(exp.TableAlias):
        aliases[alias.name] = alias.parent.name
        print(f"alias: {alias.name}, parent: {alias.parent.name}")
    for clause in scope.find_all(exp.Where):
        print(f"where: {clause}")
        filter_columns = []
        for col in clause.find_all(exp.Column):
            print(f"col: {col.name}, table_alias: {col.table}, table: {aliases[col.table]}")
            filter_columns.append({
                "column": col.name,
                "table_alias": col.table,
                "table": aliases.get(col.table)
            })
        preds.append(
            {
                "where_clause": clause.sql(),
                "filter_columns": filter_columns
            }
        )

In [ ]:
preds

In [ ]:

def filter_predicates(sql):
    optimized = optimize(parse_one(sql, dialect=Snowflake), dialect=Snowflake)
    scopes = traverse_scope(optimized)
    print(f"number of scopes: {len(scopes)}")
    preds = []
    for scope in traverse_scope(optimized):
        # print(f"scope: {scope}")
        aliases = {}
        for alias in scope.find_all(exp.TableAlias):
            aliases[alias.name] = alias.parent.name
            print(f"alias: {alias.name}, parent: {alias.parent.name}")
        for clause in scope.find_all(exp.Where):
            print(f"where: {clause}")
            filter_columns = []
            for col in clause.find_all(exp.Column):
                print(f"col: {col.name}, table_alias: {col.table}, table: {aliases[col.table]}")
                filter_columns.append({
                    "column": col.name,
                    "table_alias": col.table,
                    "table": aliases.get(col.table)
                })
            preds.append(
                {
                    "where_clause": clause.sql(),
                    "filter_columns": filter_columns
                }
            )
    return preds

f = filter_predicates(sql)

print(f)

In [ ]:
from sqlglot.optimizer.scope import traverse_scope

# ast = sqlglot.parse_one(sql, dialect=Snowflake)
optimized = optimize(parse_one(sql, dialect=Snowflake), dialect=Snowflake)
print(optimized.sql(pretty=True))
ast = optimized
physical_columns = [
    c
    for scope in traverse_scope(ast)
    for c in scope.columns
    if isinstance(scope.sources.get(c.table), exp.Table)
]
print("======")
print(physical_columns)

In [ ]:
physical_columns

In [ ]:
a = node.downstream[0].downstream[0]

In [ ]:
parsed = parse_one(sql)

for j in parsed.find_all(exp.Where):
    print(f"where: {j}")
    print(repr(j))

In [ ]:
type(node)

In [ ]:
j.find_all(exp.Predicate)

In [ ]:
j.flatten()

In [ ]:
for i in j.flatten():
  print(type(i))
  print(i)
    
for table in j.find_all(exp.Column):
  print(f"table: {table.name}")

    # for cond in j.find_all(exp.Predicate):
    #   print(repr(cond))
    #   print(f"cond: {type(cond)}")
    # print(f"join on: {j.on}")

In [ ]:
for e in j.find_all(exp.Column):
    print("hello")
    print(e)

In [ ]:
j